En este notebook se busca resolver un problema molecular sencillo utilizando el método VQE, los resultados a obtener son la energía del estado fundamental y su estado fundamental correspondiente.

En este primer bloque de código inicializamos el hamiltoniano a resolver, y el ansatz a utiilzar.

In [12]:
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit.circuit.library import EfficientSU2
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_algorithms.optimizers import COBYLA
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit.visualization import array_to_latex
#Introducimos el problema físico, en este caso se trara de una molécula de hidrógeno
molecule = MoleculeInfo(["H", "H"], [(0.0, 0.0, -0.37), (0.0, 0.0, 0.37)], charge=0, multiplicity=1)

driver = PySCFDriver.from_molecule(molecule, basis="sto3g")
problem = driver.run()

transformer = FreezeCoreTransformer()

# and you also apply transformers explicitly
problem = transformer.transform(problem)

secqop = problem.hamiltonian.second_q_op()
mapper = JordanWignerMapper()
qubit_op = mapper.map(secqop)

#Determinar el ansatz
ansatz = EfficientSU2(num_qubits=4, reps=1, entanglement="linear",
insert_barriers = True)
ansatz.decompose().draw("mpl")



MissingOptionalLibraryError: "The 'pyscf' library is required to use 'PySCFDriver'.  See https://pyscf.org/install.html."

In [11]:
import pyscf


In [1]:
import qiskit_aer

A continuación expresamos el problema como un VQE, qiskit tiene una librería que se encarga de ello directamente

In [2]:
import numpy as np
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator

# Se puede también introducir un estado_inicial aleaotorio, pero no entiendo muy bien para que sirve
vqe = VQE(Estimator(), ansatz, COBYLA())

result = vqe.compute_minimum_eigenvalue(qubit_op)
print(result)


{   'aux_operators_evaluated': None,
    'cost_function_evals': 481,
    'eigenvalue': -1.8318636403697723,
    'optimal_circuit': <qiskit.circuit.library.n_local.efficient_su2.EfficientSU2 object at 0x7ffa6e9a5180>,
    'optimal_parameters': {   ParameterVectorElement(θ[7]): 3.1416370789238264,
                              ParameterVectorElement(θ[8]): 3.141611847381526,
                              ParameterVectorElement(θ[9]): 3.1414071339674847,
                              ParameterVectorElement(θ[10]): 1.5207913244446918,
                              ParameterVectorElement(θ[11]): 1.5707300291836954,
                              ParameterVectorElement(θ[12]): -2.1786778174759007,
                              ParameterVectorElement(θ[13]): 3.04516601973528,
                              ParameterVectorElement(θ[14]): -0.1303821967336371,
                              ParameterVectorElement(θ[15]): -2.3701309471029126,
                              ParameterVectorElement(θ[0]

El result, tiene muchos parametros de return, se pueden obserar todos al hacer un print(result), darse cuenta que el estado fundamental no es uno de los parámetros que viene, pero se puede obtener este estado a partir de los parámetros del ansatz

Problemas sencillos, se pueden resolver de forma exacta utilizando la función minimum_eigenvlue, además, también se devuelve el estado fundamental 

In [3]:
from qiskit_algorithms import NumPyMinimumEigensolver
solver2 = NumPyMinimumEigensolver()
result2 = solver2.compute_minimum_eigenvalue(qubit_op)
#print(result2.eigenstate)
result2.eigenstate.draw(output='latex', prefix='R|01>=')

<IPython.core.display.Latex object>

Se puede hacer una comprobación sencilla:

In [4]:
print("El valor esperado para el estado |010> es:", result2.eigenstate.expectation_value(qubit_op))

El valor esperado para el estado |010> es: (-1.8523881735695829+0j)


Una forma de obtener el estado fundamental a partir de los parámetros es la siguiente:

In [6]:
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import Statevector
from qiskit import QuantumCircuit
optimal_points=result.optimal_parameters
param_values = [value for value in optimal_points.values()]

ansatz = EfficientSU2(num_qubits=4, reps=1, entanglement="linear", insert_barriers=True)


parameters = param_values # Por ejemplo, una lista de valores para los parámetros

# Asigna los valores de los parámetros al ansatz
bound_ansatz = ansatz.assign_parameters(parameters)

# Puedes obtener el circuito resultante con los valores asignados
bound_circuit = bound_ansatz

# Dibujar el circuito resultante
qc2=QuantumCircuit(4,4)
qc2=bound_circuit.decompose()
sv2 = Statevector(qc2)
sv2.draw(output='latex', prefix='Estado=')
#qc2.measure_all()
#qc2.draw('mpl')

<IPython.core.display.Latex object>

In [70]:
print("El valor esperado para el estado |010> es:", sv2.expectation_value(qubit_op))

El valor esperado para el estado |010> es: (-1.8523881701480942+0j)


Conclusión, el resultado obtenido realizando el VQE no es exacto, esto a priori puede ser un gran inconveniente debido a que un caso sencillo no lo resuelve bien, tengo que seguir investigando para mejorar este modelo